# Improved Baseline

Улучшенное базовое решение.

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path

In [ ]:
data_path = Path("./data")
images_path = data_path / "competition/competition/"
masks_path  = data_path / "masks/"

In [ ]:
from utils import *

In [ ]:
def get_improved_baseline_mask(image, min_treshold: float=0.17, max_treshold: float=0.4, verbose: int=0):
    # Выделение внутренней части мозга.
    #brain_mask, mask_bone, mask_void = get_brain_mask(image, return_all=True)
    brain_mask = get_brain_mask(image)
    brain_area = np.sum(brain_mask)
    
    #distance = cv2.distanceTransform((1 - mask_void.astype(np.uint8)), cv2.DIST_L1, 5)
    #distance = np.minimum(distance, 20.0) / 20.0
    
    image = np.minimum(image, brain_mask)
    smoothed_image = cv2.bilateralFilter(image, 9, 0.1, 75)
    brain_pixels = smoothed_image[brain_mask > 0.0]
    
    min_treshold = np.percentile(brain_pixels, 99)

    # Выделение средних по интенсивности пикселей.
    _, mask = cv2.threshold(image, min_treshold, 1.0, cv2.THRESH_BINARY)
    
    max_steps = 100
    for step in range(max_steps):
        # Текущая площадь, занимаемая маской, в отношении к площади мозга.
        area_to_brain_ratio = np.sum(mask) / brain_area
        
        # Выбор порога.
        delta_percent = 0.5 + 0.5 * np.exp(-step / 1.0)
        percentile_multiplier = 0.99 - 0.01 * np.exp(-step / 2.0)
        lower_bound = 0.15 + 0.03 * np.exp(-step / 2.0)
        upper_bound = 0.17 + 0.10 * np.exp(-step / 2.0)

        current_percent = 100.0 * (1.0 - area_to_brain_ratio)
        new_percentile  = percentile_multiplier * np.percentile(brain_pixels, current_percent - delta_percent)
        spread_treshold = min(upper_bound, max(lower_bound, new_percentile))
        
        if verbose >= 1:
            print(f"{step}: th: {spread_treshold:.4f}, ma/ba: {area_to_brain_ratio:.3f}")
            #plot_with_mask(smoothed_image, mask)
            
        _, mask_min = cv2.threshold(image, spread_treshold, 1.0, cv2.THRESH_BINARY)
        
        dilate_on_step_size = 15
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dilate_on_step_size, dilate_on_step_size))
        new_mask = cv2.dilate(mask, kernel, iterations=1)
        
        new_mask = np.minimum(mask_min, new_mask)
        
        kernel_size = int(3.0 * np.exp(-step / 1.0))
        if kernel_size:
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
            new_mask = cv2.morphologyEx(new_mask, cv2.MORPH_CLOSE, kernel)
        
        kernel_size = max(2, int(10.0 * np.exp(-step / 5.0)))
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
        new_mask = cv2.morphologyEx(new_mask, cv2.MORPH_OPEN, kernel)
        
        if np.sum(np.abs(new_mask - mask)) <= 1e-6 * np.sum(mask):
            mask = new_mask
            break
        
        mask = new_mask
        smoothed_image = 0.9 * smoothed_image + 0.1 * image
        brain_pixels = smoothed_image[brain_mask > 0.0]
        
    if area_to_brain_ratio > 0.2 or area_to_brain_ratio < 0.002:
        return get_baseline_mask(image)
    
    # Удаление мелких деталей.
    #kernel_size = 3
    #kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    #mask = cv2.dilate(mask, kernel, iterations=1)
    
    kernel_size = 5
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    return mask

In [ ]:
#for index in range(15):
for index in range(26600, 26725):
    print(index)
    name = f"{index:06d}.jpg"
    image_path = images_path / name
    
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    baseline_mask = np.minimum(get_brain_mask(image), get_baseline_mask(image)).astype(bool)
    mask = get_improved_baseline_mask(image, verbose=1)
    #plot_with_mask(image, mask)
    
    fig, axes = plt.subplots(1, 2, figsize=(2*8, 8))
    
    axes[0].imshow(np.maximum(image, baseline_mask))
    axes[1].imshow(np.maximum(image, mask))
    
    plt.show()
    plt.close()

In [ ]:
ious = []

for index in range(5):
    name = f"{index:06d}.jpg"
    image_path = images_path / name
    mask_path  = masks_path  / name
    
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    true_mask = cv2.imread(str(mask_path))
    true_mask = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    true_mask = cv2.normalize(true_mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    _, true_mask = cv2.threshold(true_mask, 0.5, 1.0, cv2.THRESH_BINARY)
    
    mask = get_improved_baseline_mask(image, verbose=1)
    #mask = np.minimum(get_brain_mask(image), get_baseline_mask(image))
    
    plot_with_mask(image, mask)
    
    intersection_over_union = np.count_nonzero(np.minimum(mask, true_mask)) / np.count_nonzero(np.maximum(mask, true_mask))
    #print(intersection_over_union)
    ious.append(intersection_over_union)

In [ ]:
sum(ious) / 5

In [ ]:
def df_from_mask(name, mask):
    numbers = pd.Series(np.arange(mask.size), name = 'ID')
    ind = numbers.apply(lambda n: name + f"_{n // mask.shape[1]}_{n % mask.shape[1]}")
    return pd.DataFrame({'value': mask.flatten()}, index = ind, dtype=np.int32)

In [ ]:
from tqdm import tqdm

dataframes = []

for index in tqdm(range(26600, 26725)):
    name = f"{index:06d}"
    image_path = images_path / (name + ".jpg")
    
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    mask = get_improved_baseline_mask(image)
    #mask = np.minimum(get_brain_mask(image), get_baseline_mask(image)).astype(bool)
    
    #answer = pd.concat([answer, df_from_mask(name, mask)])
    dataframes.append(df_from_mask(name, mask))
    
    #fig = plt.figure(figsize=(10, 10))
    #plt.imshow(mask)
    #plt.show()

In [ ]:
answer = pd.concat(dataframes)
answer.to_csv('improved_baseline.csv')